In [4]:
sc.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("coming~").setMaster("spark://10.120.26.200:7077").set("spark.executor.memory", '16g').set('spark.cores.max', '4')
sc = SparkContext(conf=conf)
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=spark://10.120.26.200:7077 appName=coming~>

In [50]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

An error was encountered:
Invalid status code '404' from http://10.120.26.200:8998/sessions/8 with error payload: {"msg":"Session '8' not found."}


In [32]:
lines = sc.textFile("hdfs://10.120.26.200/user/spark/spark101/spark203/1101.TW.csv")
header = lines.first()
data = lines.filter(lambda line:line!=header)
data.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2000-01-04,17.805401,18.226801,17.805401,18.226801,7.236699,97168521223', '2000-01-05,17.858000,19.174999,17.858000,18.700899,7.424934,149925508353', '2000-01-06,18.542900,18.753599,17.752701,17.910700,7.111196,74480726535', '2000-01-07,17.173201,17.963400,16.857100,17.858000,7.090272,87599471028', '2000-01-10,17.910700,18.542900,17.647301,18.437500,7.320355,78718785880', '2000-01-11,19.017000,19.227699,17.542000,17.910700,7.111196,78772033842', '2000-01-12,17.805401,18.121401,17.436600,17.436600,6.922960,41878878974', '2000-01-13,17.647301,18.437500,17.383900,17.963400,7.132120,91567972662', '2000-01-14,18.068800,18.226801,17.383900,17.594601,6.985693,38590181', '2000-01-17,17.963400,18.542900,17.752701,18.226801,7.236699,66482427']

In [33]:
tmp = data.map(lambda x:x.split(','))
df = tmp.toDF(header.split(','))
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------+---------+---------+---------+------------+
|      Date|     Open|     High|      Low|    Close|Adj Close|      Volume|
+----------+---------+---------+---------+---------+---------+------------+
|2000-01-04|17.805401|18.226801|17.805401|18.226801| 7.236699| 97168521223|
|2000-01-05|17.858000|19.174999|17.858000|18.700899| 7.424934|149925508353|
|2000-01-06|18.542900|18.753599|17.752701|17.910700| 7.111196| 74480726535|
|2000-01-07|17.173201|17.963400|16.857100|17.858000| 7.090272| 87599471028|
|2000-01-10|17.910700|18.542900|17.647301|18.437500| 7.320355| 78718785880|
|2000-01-11|19.017000|19.227699|17.542000|17.910700| 7.111196| 78772033842|
|2000-01-12|17.805401|18.121401|17.436600|17.436600| 6.922960| 41878878974|
|2000-01-13|17.647301|18.437500|17.383900|17.963400| 7.132120| 91567972662|
|2000-01-14|18.068800|18.226801|17.383900|17.594601| 6.985693|    38590181|
|2000-01-17|17.963400|18.542900|17.752701|18.226801| 7.236699|    66482427|
|2000-01-18|

In [34]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


DF = df.select(df.Date, df.Low, df.High, df.Close)
DF = DF.withColumn("ID", 
                   F.row_number().over(Window.orderBy("Date"))
     )

DF.createOrReplaceTempView("test123")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
l="""
SELECT 
    Date,
    Close,
    Low,
    High,
    MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING) AS min,
    MAX(High) OVER (ORDER BY Date ASC ROWS 8 PRECEDING) AS max,
    (Close-MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING))/(MAX(High) OVER (ORDER BY Date ASC ROWS 8 PRECEDING)-MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING)) as RSV,
    ID
FROM   test123
"""


rback = spark.sql(l)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
rback.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------+---------+---------+---------+-------------------+---+
|      Date|    Close|      Low|     High|      min|      max|                RSV| ID|
+----------+---------+---------+---------+---------+---------+-------------------+---+
|2000-01-04|18.226801|17.805401|18.226801|17.805401|18.226801|                1.0|  1|
|2000-01-05|18.700899|17.858000|19.174999|17.805401|19.174999| 0.6538400318925699|  2|
|2000-01-06|17.910700|17.752701|18.753599|17.752701|19.174999|0.11108712801396055|  3|
|2000-01-07|17.858000|16.857100|17.963400|16.857100|19.174999| 0.4318134655565239|  4|
|2000-01-10|18.437500|17.647301|18.542900|16.857100|19.174999| 0.6818243590423916|  5|
|2000-01-11|17.910700|17.542000|19.227699|16.857100|19.227699| 0.4444446319263606|  6|
|2000-01-12|17.436600|17.436600|18.121401|16.857100|19.227699|0.24445298424575349|  7|
|2000-01-13|17.963400|17.383900|18.437500|16.857100|19.227699| 0.4666753002089345|  8|
|2000-01-14|17.594601|17.383900|18.226801|1

In [37]:
rbackRDD = rback.rdd.map(tuple)
drop_rows = rbackRDD.take(7)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
def drop_rowsF(line):
    if line in drop_rows:
        return False
    else:
        return True

# rbackRDD1 = rbackRDD.filter(lambda line: line != drop_rows)drop_rowsF
rbackRDD1 = rbackRDD.filter(drop_rowsF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
rbackRDD1.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('2000-01-13', '17.963400', '17.383900', '18.437500', '16.857100', '19.227699', 0.4666753002089345, 8), ('2000-01-14', '17.594601', '17.383900', '18.226801', '16.857100', '19.227699', 0.3111032274965108, 9), ('2000-01-17', '18.226801', '17.752701', '18.542900', '16.857100', '19.227699', 0.5777868800248367, 10), ('2000-01-18', '17.752701', '17.700001', '18.226801', '16.857100', '19.227699', 0.3777952323442296, 11), ('2000-01-19', '17.805401', '17.647301', '18.332100', '16.857100', '19.227699', 0.40002590062680354, 12), ('2000-01-20', '19.017000', '18.016100', '19.017000', '17.383900', '19.227699', 0.8857256132582773, 13), ('2000-01-21', '19.912500', '19.438400', '20.333900', '17.383900', '20.333900', 0.8571525423728819, 14), ('2000-01-24', '20.913401', '19.912500', '21.229500', '17.383900', '21.229500', 0.9178024235489908, 15), ('2000-01-25', '20.492001', '20.070499', '20.755400', '17.383900', '21.229500', 0.808222644060744, 16), ('2000-01-26', '21.387501', '20.386600', '21.387501', '1